<a href="https://colab.research.google.com/github/Tiabet/DACON_WebClick/blob/main/WebClick_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.7 MB/s eta 0:00:00


In [5]:
import duckdb

In [14]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import plotly.express as px
import lightgbm as lgb
import category_encoders as ce
from sklearn.ensemble import AdaBoostClassifier

In [2]:
# 동일한 결과 보장을 위해 Seed값을 고정합니다
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed를 42로 고정

In [6]:
train_path = 'drive/MyDrive/data/train.csv'

con = duckdb.connect()

train = con.query(f"""(SELECT *
                        FROM read_csv_auto('{train_path}')
                        WHERE Click = 0
                        ORDER BY random()
                        LIMIT 30000)
                        UNION ALL
                        (SELECT *
                        FROM read_csv_auto('{train_path}')
                        WHERE Click = 1
                        ORDER BY random()
                        LIMIT 30000)""").df()


con.close()

train.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,ID,Click,F01,F02,F03,F04,F05,F06,F07,F08,...,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39
0,TRAIN_16605215,0,JCDXFYU,PILDDJU,IAGJDOH,13.0,LFPUEOV,179,KGBFNSR,FTPHMPQ,...,NMICAKX,KHZNEZF,1386.0,1.0,QMOULXS,IRUDRFB,11.0,NNWXELZ,0.0,KXZUMSP
1,TRAIN_28216154,0,GZVXJZK,HJHTEBK,DTZFPRW,100.0,AUMSXVU,0,PQZBVMG,OFKQGTY,...,PEGBZUB,GTISJWW,4121.0,0.0,WJZCMAZ,IRUDRFB,4.0,TFJMLCZ,0.0,MVOGQRB
2,TRAIN_00075550,0,VNOHLIR,PKLDGLX,IAGJDOH,12.0,EVTUBMN,1,XWCMPCY,LPYPUNA,...,TAYLVVM,ENBEWZP,36.0,0.0,SHMKPOR,IRUDRFB,1.0,NATXQIW,0.0,FELYDVW
3,TRAIN_25361440,0,NTKWMRP,QFPSNDM,PQGWFJZ,9.0,WLESRLK,69,FCGGFDO,SODVHOK,...,WWWEXSL,GTISJWW,1640.0,0.0,GJFOPKA,IRUDRFB,4.0,AUGTURV,1.0,WCESWZK
4,TRAIN_16069022,0,JCDXFYU,PILDDJU,IAGJDOH,NaN,LFPUEOV,0,ZQDOVZM,VAWXMCR,...,ANXDRDA,KHZNEZF,8690.0,NaN,QMOULXS,IRUDRFB,NaN,UUAJCBU,0.0,TFBWUZU


In [9]:
len(train)

60000

In [2]:
# 제공된 train 데이터와 test 데이터를 불러옵니다
train = pd.read_csv('drive/MyDrive/data/train.csv')
test = pd.read_csv('drive/MyDrive/data/test.csv')
sample_submission = pd.read_csv('drive/MyDrive/data/sample_submission.csv')

In [3]:
train.to_parquet('train.parquet', compression = 'gzip', engine = 'pyarrow', index = False)

In [4]:
#EDA 1 : Sparse and Dense
train.head()

,ID,Click,F01,F02,F03,F04,F05,F06,F07,F08,...,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39
0,TRAIN_00000000,1,NSLHFNS,AVKQTCL,DTZFPRW,114.0,ISVXFVA,1,PQZBVMG,LPYPUNA,...,NZGEZLW,GTISJWW,380.0,2.0,AXQFZWC,IRUDRFB,NaN,TFJMLCZ,0.0,AURZYDY
1,TRAIN_00000001,0,VGIVWZQ,LSUSMVO,PQGWFJZ,26.0,NFRVLWS,43,IMPIGJT,MIGYEEG,...,NZGEZLW,GTISJWW,466.0,1.0,DRVVDHZ,IRUDRFB,19.0,AUGTURV,0.0,LUZRMLU
2,TRAIN_00000002,0,JCDXFYU,PILDDJU,IAGJDOH,119.0,LFPUEOV,0,FFUTIRZ,OFKQGTY,...,VHXETCF,KHZNEZF,197.0,0.0,QMOULXS,IRUDRFB,8.0,ZVSTLNM,0.0,MHBRSQK
3,TRAIN_00000003,1,PSMFWTP,ZYAVJHP,NaN,15.0,ATQPZSJ,26,ZDTZNSB,THBWWCD,...,IVIRTPR,GTISJWW,8640.0,0.0,IZLJUJS,IRUDRFB,14.0,ZBSRLCQ,0.0,GAZBSSZ
4,TRAIN_00000004,0,SLCRICD,QPQWGXA,NaN,13.0,CHZGJZR,20,PQZBVMG,MIGYEEG,...,NZGEZLW,WHSRKIM,41774.0,0.0,BHBIZCL,IRUDRFB,13.0,QHYLSBX,0.0,QTATWAY


In [5]:
len(test)

4538541

In [6]:
len(sample_submission)

7539172

In [7]:
#EDA 2 : Imbalance
click = train['Click'].value_counts(normalize=True)

click_figure = px.bar(click,
             x=['Not Clicked : 0', 'Clicked : 1'],
             y=click.values.tolist(),
             labels={'x': 'Value', 'y': 'Percentage'},
             width = 450,
             height = 500
            )

# 그래프 표시
click_figure.show()

In [8]:
#Data Preprocessing 1 : Select x, y
train_x = train.drop(columns=['ID', 'Click'])
train_y = train['Click']

test_x = test.drop(columns=['ID'])

In [9]:
#Data Preprocessing 2 : Fill NaN
for col in tqdm(train_x.columns):
    if train_x[col].isnull().sum() != 0:
        train_x[col].fillna(0, inplace=True)
        test_x[col].fillna(0, inplace=True)

100%|██████████| 39/39 [00:02<00:00, 14.07it/s]


In [10]:
#Data Preprocessing 3 : Count Encoding
encoding_target = list(train_x.dtypes[train_x.dtypes == "object"].index)

enc = ce.CountEncoder(cols = encoding_target).fit(train_x, train_y)
X_train_encoded = enc.transform(train_x)
X_test_encoded = enc.transform(test_x)

In [11]:
#Model Setting
model = AdaBoostClassifier()
#Model Train and Inference
model.fit(X_train_encoded, train_y)

AdaBoostClassifier()

In [15]:
pred = model.predict_proba(X_test_encoded)
display(model.classes_)
display(pred)

array([0, 1])

array([[0.49746058, 0.50253942],
       [0.50190728, 0.49809272],
       [0.49980626, 0.50019374],
       ...,
       [0.50389745, 0.49610255],
       [0.50069486, 0.49930514],
       [0.49995004, 0.50004996]])

In [18]:
#Submission
sample_submission = pd.read_csv('drive/MyDrive/data/sample_submission.csv')
sample_submission

sample_submission['Click'] = pred[:,1]
sample_submission

sample_submission.to_csv('drive/MyDrive/data/baseline_submission.csv', index=False)

lgbm

In [15]:
# Split the dataset into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train_encoded, train_y, test_size=0.2, random_state=42)

In [16]:
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)

In [20]:
# Set the parameters for the LightGBM model
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': ['binary_logloss', 'auc'],
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

# Train the model
num_round = 20000
bst = lgb.train(params, train_data, num_round, valid_sets=[valid_data],callbacks=[lgb.early_stopping(stopping_rounds=10)],)


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[212]	valid_0's binary_logloss: 0.612448	valid_0's auc: 0.724198


In [24]:
# Predict on the test set
y_pred_test = bst.predict(X_test_encoded, num_iteration=bst.best_iteration)


In [26]:
sample_submission['Click'] = y_pred_test
sample_submission

,ID,Click
0,TEST_0000000,0.614827
1,TEST_0000001,0.513895
2,TEST_0000002,0.438905
3,TEST_0000003,0.746767
4,TEST_0000004,0.821814
...,...,...
4538536,TEST_4538536,0.442299
4538537,TEST_4538537,0.709947
4538538,TEST_4538538,0.227438
4538539,TEST_4538539,0.464717


In [27]:
sample_submission.to_csv("drive/MyDrive/data/lgbm_prediction.csv",index = False)

automl 사용

In [29]:
pip install flaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 5.5 MB/s eta 0:00:00


In [34]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 16.1 MB/s eta 0:00:00


In [30]:
from flaml import AutoML

In [41]:
# Initialize AutoML
automl = AutoML()

# Specify the settings
automl_settings = {
    "time_budget": 300,  # Total time budget in seconds
    "metric": 'roc_auc',  # Evaluation metric
    "task": 'classification',  # Task type
    "log_file_name": 'automl.log',  # Log file
    "estimator_list": ['lgbm', 'xgboost', 'catboost', 'rf', 'extra_tree'],  # List of estimators to use
    "eval_method": "cv",  # Cross-validation method
    "n_splits": 5,  # Number of cross-validation splits
    "early_stop": 10
}

In [42]:
# Train the model
automl.fit(X_train=X_train_encoded, y_train=train_y, **automl_settings)

[flaml.automl.logger: 05-20 12:49:36] {1680} INFO - task = classification
[flaml.automl.logger: 05-20 12:49:36] {1691} INFO - Evaluation method: cv
[flaml.automl.logger: 05-20 12:49:36] {1789} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl.logger: 05-20 12:49:36] {1901} INFO - List of ML learners in AutoML Run: ['lgbm', 'xgboost', 'catboost', 'rf', 'extra_tree']
[flaml.automl.logger: 05-20 12:49:36] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 05-20 12:49:38] {2345} INFO - Estimated sufficient time budget=16835s. Estimated necessary time budget=36s.
[flaml.automl.logger: 05-20 12:49:38] {2392} INFO -  at 2.0s,	estimator lgbm's best error=0.3493,	best estimator lgbm's best error=0.3493
[flaml.automl.logger: 05-20 12:49:38] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 05-20 12:49:38] {2392} INFO -  at 2.6s,	estimator lgbm's best error=0.3493,	best estimator lgbm's best error=0.3493
[flaml.automl.logger: 05-20 12:49:38] {2219} INF

In [43]:
print("Best model:", automl.model.estimator)

Best model: LGBMClassifier(colsample_bytree=0.5764187075082378,
               learning_rate=0.047513351479242745, max_bin=255,
               min_child_samples=6, n_estimators=1, n_jobs=-1, num_leaves=20,
               reg_alpha=0.007704104902643932, reg_lambda=0.010770998670064029,
               verbose=-1)


In [44]:
# Predict probabilities on the test set
y_pred_test_proba = automl.predict_proba(X_test_encoded)[:, 1]

# Output the probabilities for test set
print(y_pred_test_proba)

[0.55143402 0.48075621 0.42414883 ... 0.13935322 0.47152536 0.55089357]


In [45]:
sample_submission['Click'] = y_pred_test
sample_submission

,ID,Click
0,TEST_0000000,0.614827
1,TEST_0000001,0.513895
2,TEST_0000002,0.438905
3,TEST_0000003,0.746767
4,TEST_0000004,0.821814
...,...,...
4538536,TEST_4538536,0.442299
4538537,TEST_4538537,0.709947
4538538,TEST_4538538,0.227438
4538539,TEST_4538539,0.464717


In [46]:
sample_submission.to_csv("drive/MyDrive/data/automl_prediction.csv",index = False)